In [ ]:
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import TimeSeriesSplit
from tqdm import tqdm

## 1. Competition Metric Implementation

In [ ]:
MIN_INVESTMENT = 0.0
MAX_INVESTMENT = 2.0

class ParticipantVisibleError(Exception):
    pass

def portfolio_score(solution: pd.DataFrame, submission: pd.DataFrame) -> float:
    """
    Calculates the competition's volatility-adjusted Sharpe ratio.
    
    Parameters:
    -----------
    solution : pd.DataFrame
        Must contain: 'forward_returns', 'risk_free_rate'
    submission : pd.DataFrame
        Must contain: 'prediction' (allocation between 0 and 2)
    
    Returns:
    --------
    float : Adjusted Sharpe ratio (capped at 1,000,000)
    """
    sol = solution.copy()
    sol['position'] = submission['prediction'].clip(MIN_INVESTMENT, MAX_INVESTMENT)

    if sol['position'].max() > MAX_INVESTMENT:
        raise ParticipantVisibleError(
            f'Position of {sol["position"].max()} exceeds maximum of {MAX_INVESTMENT}'
        )
    
    if sol['position'].min() < MIN_INVESTMENT:
        raise ParticipantVisibleError(
            f'Position of {sol["position"].min()} below minimum of {MIN_INVESTMENT}'
        )

    # Strategy returns = risk_free_rate on cash + forward_returns on invested portion
    sol['strategy_returns'] = (
        sol['risk_free_rate'] * (1.0 - sol['position']) +
        sol['position'] * sol['forward_returns']
    )

    # Calculate strategy's Sharpe ratio
    strategy_excess_returns = sol['strategy_returns'] - sol['risk_free_rate']
    strategy_excess_cumulative = (1 + strategy_excess_returns).prod()
    strategy_mean_excess_return = strategy_excess_cumulative ** (1 / len(sol)) - 1
    strategy_std = sol['strategy_returns'].std()

    trading_days_per_yr = 252
    if strategy_std == 0:
        raise ZeroDivisionError('Zero volatility')
    
    sharpe = strategy_mean_excess_return / strategy_std * np.sqrt(trading_days_per_yr)
    strategy_volatility = float(strategy_std * np.sqrt(trading_days_per_yr) * 100)

    # Calculate market return and volatility
    market_excess_returns = sol['forward_returns'] - sol['risk_free_rate']
    market_excess_cumulative = (1 + market_excess_returns).prod()
    market_mean_excess_return = market_excess_cumulative ** (1 / len(sol)) - 1
    market_std = sol['forward_returns'].std()
    market_volatility = float(market_std * np.sqrt(trading_days_per_yr) * 100)

    # Volatility penalty: kicks in above 120% of market volatility
    excess_vol = max(0.0, (strategy_volatility / market_volatility) - 1.2) if market_volatility > 0 else 0
    vol_penalty = 1.0 + excess_vol

    # Return penalty: kicks in if we underperform market
    return_gap = max(
        0.0,
        (market_mean_excess_return - strategy_mean_excess_return) * 100 * trading_days_per_yr
    )
    return_penalty = 1.0 + (return_gap ** 2) / 100

    # Adjust Sharpe by penalties
    adjusted_sharpe = sharpe / (vol_penalty * return_penalty)
    
    return min(float(adjusted_sharpe), 1_000_000)

## 2. Time-Series Cross-Validation

In [ ]:
def time_series_cv(predict_fn, train_df: pd.DataFrame, n_splits=10, test_size=180):
    """
    Perform time-series cross-validation for allocation models.
    
    Parameters:
    -----------
    predict_fn : callable
        Function that takes (train_fold_df, test_fold_df) and returns predictions array
    train_df : pd.DataFrame
        Training data with all features + targets
    n_splits : int
        Number of CV folds
    test_size : int
        Size of test fold (in days)
    
    Returns:
    --------
    dict : {'mean_score', 'std_score', 'fold_scores', 'oof_predictions'}
    """
    tscv = TimeSeriesSplit(n_splits=n_splits, test_size=test_size)
    
    fold_scores = []
    oof_predictions = np.full(len(train_df), np.nan)
    
    for fold, (train_idx, test_idx) in enumerate(tqdm(tscv.split(train_df), total=n_splits)):
        train_fold = train_df.iloc[train_idx].copy()
        test_fold = train_df.iloc[test_idx].copy()
        
        # Get predictions
        predictions = predict_fn(train_fold, test_fold)
        
        # Score this fold
        solution = test_fold[['forward_returns', 'risk_free_rate']].copy()
        submission = pd.DataFrame({'prediction': predictions})
        
        try:
            score = portfolio_score(solution, submission)
            fold_scores.append(score)
            oof_predictions[test_idx] = predictions
            print(f"Fold {fold+1}/{n_splits}: Score = {score:.4f}")
        except Exception as e:
            print(f"Fold {fold+1}/{n_splits}: ERROR - {e}")
            fold_scores.append(0.0)
    
    mean_score = np.mean(fold_scores)
    std_score = np.std(fold_scores)
    
    print(f"\n{'='*60}")
    print(f"Mean CV Score: {mean_score:.4f} ± {std_score:.4f}")
    print(f"{'='*60}")
    
    return {
        'mean_score': mean_score,
        'std_score': std_score,
        'fold_scores': fold_scores,
        'oof_predictions': oof_predictions
    }

## 3. Test with Constant Allocation Baseline

In [ ]:
# Load data
DATA_PATH = Path('./hull-tactical-market-prediction')
train = pd.read_csv(DATA_PATH / 'train.csv')

print(f"Train data shape: {train.shape}")
print(f"Date range: {train['date_id'].min()} to {train['date_id'].max()}")
print(f"Missing values per column:\n{train.isnull().sum().sum()} total")

In [ ]:
# Test 1: Constant allocation = 0.8
def constant_08_predict(train_fold, test_fold):
    return np.full(len(test_fold), 0.8)

results_08 = time_series_cv(constant_08_predict, train, n_splits=5, test_size=180)

In [ ]:
# Test 2: Constant allocation = 1.0 (track market)
def constant_10_predict(train_fold, test_fold):
    return np.full(len(test_fold), 1.0)

results_10 = time_series_cv(constant_10_predict, train, n_splits=5, test_size=180)

In [ ]:
# Compare baselines
print("\nBaseline Comparison:")
print(f"Constant 0.8: {results_08['mean_score']:.4f} ± {results_08['std_score']:.4f}")
print(f"Constant 1.0: {results_10['mean_score']:.4f} ± {results_10['std_score']:.4f}")

## 4. Visualization Helper

In [ ]:
def plot_cv_results(results_dict):
    """
    Plot fold scores and prediction distribution.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 4))
    
    # Fold scores
    axes[0].plot(results_dict['fold_scores'], marker='o', linewidth=2)
    axes[0].axhline(results_dict['mean_score'], color='r', linestyle='--', label='Mean')
    axes[0].fill_between(
        range(len(results_dict['fold_scores'])),
        results_dict['mean_score'] - results_dict['std_score'],
        results_dict['mean_score'] + results_dict['std_score'],
        alpha=0.2, color='r'
    )
    axes[0].set_xlabel('Fold')
    axes[0].set_ylabel('Score')
    axes[0].set_title('Score by CV Fold')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Prediction distribution
    oof = results_dict['oof_predictions']
    oof = oof[~np.isnan(oof)]
    axes[1].hist(oof, bins=50, edgecolor='black', alpha=0.7)
    axes[1].axvline(oof.mean(), color='r', linestyle='--', label=f'Mean: {oof.mean():.3f}')
    axes[1].set_xlabel('Allocation')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('OOF Prediction Distribution')
    axes[1].legend()
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Example
plot_cv_results(results_08)

---
## ✅ You now have:
1. Proper metric implementation
2. Time-series CV framework
3. Baseline scores to beat

**Next:** Use this framework to validate your models in the next notebooks!